Deep Learning with Tensorflow
=============

Assignment II
------------

During one of the lectures in [Lab 1](https://deep-learning-su.github.io/labs/lab-1/) we trained fully connected network to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters. 

The goal of this assignment is make the neural network convolutional.

For this exercise, you would need the `notMNIST.pickle` created in `Lab 1`. You can obtain it by rerunning the given paragraphs without having to solve the problems (although it is highly recommended to do it if you haven't already).

In [27]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [28]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [29]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [30]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## Problem 1
Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

Edit the snippet bellow by changing the `model` function.

### 1.1 - Define the model
Implement the `model` function bellow. Take a look at the following TF functions:
- **tf.nn.conv2d(X,W1, strides = [1,s,s,1], padding = 'SAME'):** given an input $X$ and a group of filters $W1$, this function convolves $W1$'s filters on X. The third input ([1,f,f,1]) represents the strides for each dimension of the input (m, n_H_prev, n_W_prev, n_C_prev). You can read the full documentation [here](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d)
- **tf.nn.relu(Z1):** computes the elementwise ReLU of Z1 (which can be any shape). You can read the full documentation [here.](https://www.tensorflow.org/api_docs/python/tf/nn/relu)

### 1.2 - Compute loss

Implement the `compute_loss` function below. You might find these two functions helpful: 

- **tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y):** computes the softmax entropy loss. This function both computes the softmax activation function as well as the resulting loss. You can check the full documentation  [here.](https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits)
- **tf.reduce_mean:** computes the mean of elements across dimensions of a tensor. Use this to sum the losses over all the examples to get the overall cost. You can check the full documentation [here.](https://www.tensorflow.org/api_docs/python/tf/reduce_mean)


In [31]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

def output_size_no_pool(input_size, filter_size, padding, conv_stride):
    if padding == 'same':
        padding = -1.00
    elif padding == 'valid':
        padding = 0.00
    else:
        return None
    output_1 = float(((input_size - filter_size - 2*padding) / conv_stride) + 1.00)
    output_2 = float(((output_1 - filter_size - 2*padding) / conv_stride) + 1.00)
    return int(np.ceil(output_2))


graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  weights_l1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  biases_l1 = tf.Variable(tf.zeros([depth]))
  
  weights_l2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth],   stddev=0.1))
  biases_l2 = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  final_image_size = output_size_no_pool(image_size, patch_size, padding='same', conv_stride=2)
  weights_l3 = tf.Variable(tf.truncated_normal([final_image_size * final_image_size * depth, num_hidden], stddev=0.1))
  biases_l3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  weights_l4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  biases_l4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, weights_l1, strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + biases_l1)
    
    conv = tf.nn.conv2d(hidden, weights_l2, strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + biases_l2)   
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, weights_l3) + biases_l3)

    return tf.matmul(hidden, weights_l4) + biases_l4

  def compute_loss(labels, logits):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = compute_loss(tf_train_labels, logits)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

### 1.3 - Measure the accuracy and tune your model

Run the snippet bellow to measure the accuracy of your model. Try to achieve a test accuracy of around 80%. Iterate on the filters size.

In [32]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.730139
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.759906
Minibatch accuracy: 50.0%
Validation accuracy: 45.1%
Minibatch loss at step 100: 0.795882
Minibatch accuracy: 62.5%
Validation accuracy: 69.9%
Minibatch loss at step 150: 1.080710
Minibatch accuracy: 62.5%
Validation accuracy: 71.5%
Minibatch loss at step 200: 0.436723
Minibatch accuracy: 93.8%
Validation accuracy: 77.8%
Minibatch loss at step 250: 0.813656
Minibatch accuracy: 75.0%
Validation accuracy: 78.4%
Minibatch loss at step 300: 1.112368
Minibatch accuracy: 62.5%
Validation accuracy: 77.9%
Minibatch loss at step 350: 1.184072
Minibatch accuracy: 68.8%
Validation accuracy: 78.8%
Minibatch loss at step 400: 0.891509
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 450: 0.388720
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Minibatch loss at step 500: 0.590548
Minibatch accuracy: 75.0%
Validation accuracy: 80.8%
Mi

---
Problem 2
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [33]:
batch_size = 16
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    weights_l1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    biases_l1 = tf.Variable(tf.zeros([depth]))
    
    weights_l2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    biases_l2 = tf.Variable(tf.constant(1.0, shape=[depth]))

    weights_l3 = tf.Variable(tf.truncated_normal([final_image_size * final_image_size * depth, num_hidden], stddev=0.1))
    biases_l3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    weights_l4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    biases_l4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv_1 = tf.nn.conv2d(data, weights_l1, strides=[1, 1, 1, 1], padding='SAME')
        hidden_1 = tf.nn.relu(conv_1 + biases_l1)
        pool_1 = tf.nn.max_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
      
        conv_2 = tf.nn.conv2d(pool_1, weights_l2, strides=[1, 1, 1, 1], padding='SAME')
        hidden_2 = tf.nn.relu(conv_2 + biases_l2)
        pool_2 = tf.nn.max_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, weights_l3) + biases_l3)
        
        return tf.matmul(hidden, weights_l4) + biases_l4

    def compute_loss(labels, logits):
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))

    # Training computation.
    logits = model(tf_train_dataset)
    loss = compute_loss(tf_train_labels, logits)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [34]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.383842
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.833165
Minibatch accuracy: 25.0%
Validation accuracy: 37.3%
Minibatch loss at step 100: 0.994763
Minibatch accuracy: 56.2%
Validation accuracy: 56.9%
Minibatch loss at step 150: 1.066578
Minibatch accuracy: 62.5%
Validation accuracy: 62.5%
Minibatch loss at step 200: 0.491663
Minibatch accuracy: 87.5%
Validation accuracy: 75.2%
Minibatch loss at step 250: 0.659681
Minibatch accuracy: 81.2%
Validation accuracy: 75.7%
Minibatch loss at step 300: 1.099446
Minibatch accuracy: 62.5%
Validation accuracy: 76.6%
Minibatch loss at step 350: 1.175911
Minibatch accuracy: 62.5%
Validation accuracy: 80.0%
Minibatch loss at step 400: 0.736692
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Minibatch loss at step 450: 0.478065
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 500: 0.514336
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%
M

---
Problem 3
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [35]:
def output_size_pool(input_size, conv_filter_size, pool_filter_size, padding, conv_stride, pool_stride):
    if padding == 'same':
        padding = -1.00
    elif padding == 'valid':
        padding = 0.00
    else:
        return None
    
    output_1 = (((input_size - conv_filter_size - 2*padding) / conv_stride) + 1.00)
    output_2 = (((output_1 - pool_filter_size - 2*padding) / pool_stride) + 1.00)    
    output_3 = (((output_2 - conv_filter_size - 2*padding) / conv_stride) + 1.00)
    output_4 = (((output_3 - pool_filter_size - 2*padding) / pool_stride) + 1.00)

    return int(output_4)

batch_size = 16
depth = 32
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    weights_l1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    biases_l1 = tf.Variable(tf.zeros([depth]))

    weights_l2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    biases_l2 = tf.Variable(tf.constant(1.0, shape=[depth]))
  
    final_image_size = output_size_pool(input_size=image_size, conv_filter_size=5, pool_filter_size=2, padding='valid', conv_stride=1, pool_stride=2)
    weights_l3 = tf.Variable(tf.truncated_normal([final_image_size * final_image_size * depth, num_hidden], stddev=0.1))
    biases_l3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    weights_l4 = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
    biases_l4 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    weights_l5 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    biases_l5 = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv_1 = tf.nn.conv2d(data, weights_l1, strides=[1, 1, 1, 1], padding='VALID')
        hidden_1 = tf.nn.relu(conv_1 + biases_l1)
        pool_1 = tf.nn.avg_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        
        conv_2 = tf.nn.conv2d(pool_1, weights_l2, strides=[1, 1, 1, 1], padding='VALID')
        hidden_2 = tf.nn.relu(conv_2 + biases_l2)
        pool_2 = tf.nn.avg_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, weights_l3) + biases_l3)
        keep_prob = 0.5
        hidden_drop = tf.nn.dropout(hidden, keep_prob)
        
        hidden_2 = tf.nn.relu(tf.matmul(hidden_drop, weights_l4) + biases_l4)
        hidden_2_drop = tf.nn.dropout(hidden_2, keep_prob)
        
        return tf.matmul(hidden_2_drop, weights_l5) + biases_l5

    def compute_loss(labels, logits):
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
        
    # Training computation.
    logits = model(tf_train_dataset)
    loss = compute_loss(tf_train_labels, logits)

    global_step = tf.Variable(0) 
    start_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [36]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.057242
Minibatch accuracy: 6.2%
Validation accuracy: 9.8%
Minibatch loss at step 50: 2.182587
Minibatch accuracy: 25.0%
Validation accuracy: 19.8%
Minibatch loss at step 100: 1.758024
Minibatch accuracy: 43.8%
Validation accuracy: 28.6%
Minibatch loss at step 150: 1.121899
Minibatch accuracy: 50.0%
Validation accuracy: 34.4%
Minibatch loss at step 200: 1.636439
Minibatch accuracy: 37.5%
Validation accuracy: 40.3%
Minibatch loss at step 250: 0.999373
Minibatch accuracy: 75.0%
Validation accuracy: 43.7%
Minibatch loss at step 300: 1.820318
Minibatch accuracy: 43.8%
Validation accuracy: 47.6%
Minibatch loss at step 350: 2.289584
Minibatch accuracy: 50.0%
Validation accuracy: 47.4%
Minibatch loss at step 400: 1.541235
Minibatch accuracy: 43.8%
Validation accuracy: 51.0%
Minibatch loss at step 450: 1.045562
Minibatch accuracy: 62.5%
Validation accuracy: 54.2%
Minibatch loss at step 500: 1.564436
Minibatch accuracy: 43.8%
Validation accuracy: 56.3%
Min